In [39]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

In [40]:
# sort the dataframe by hometeams
df = pd.read_csv('merged.csv')
df_label = pd.read_csv('labels.csv')
df_awaydict = pd.read_csv("awaydict.csv")
df_homedict = pd.read_csv("homedict.csv")
df.sort_values(by=['HomeTeam'], ascending=True)
#preprocessing data 
selected_features = ['HomeTeam', 'AwayTeam','FTHG','FTAG','FTR','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']
df = df[selected_features].dropna()
homedict = dict(zip(df_homedict['HomeTeam'], df_homedict['ID']))
awaydict = dict(zip(df_awaydict['AwayTeam'], df_awaydict['ID']))
df["HomeTeam"] = df["HomeTeam"].map(homedict)
df["AwayTeam"] = df["AwayTeam"].map(awaydict)

print(df)




      HomeTeam  AwayTeam  FTHG  FTAG FTR HTR  HS  AS  HST  AST  HF  AF  HC  \
0            1       100     2     1   H   D  14   4    6    2  13  19   9   
1            2       101     2     2   D   A  11  13    3    3  16  10   3   
2            3       102     1     2   A   A  14   5    5    4  14  20   4   
3            4       103     0     1   A   D  19  11    6    4  10  10   8   
4            5       104     0     1   A   D  12   7    2    2  14   9   2   
...        ...       ...   ...   ...  ..  ..  ..  ..  ...  ...  ..  ..  ..   
2695        13       117     1     0   H   H   6  10    3    4  12  22   3   
2696         2       107     4     1   H   D  12   7    6    2  10   5   3   
2697         8       127     3     1   H   D  29   7    8    5   6   3   5   
2698        20       104     3     2   H   A  24   4    5    2   5  11  13   
2699        22       106     0     5   A   A   9  19    0   13  13   7   3   

      AC  HY  AY  HR  AR  
0      3   2   2   0   1  
1      6 

In [48]:
import pandas as pd
#normalizing our data
scaler = StandardScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(df[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']]))

# assign weights to each parameter (full-timegoals, shots, shots on target, fouls,corners, yellow cards, red cards )
weights = [0.25, 0.15, 0.2, 0.1, 0.1, 0.1, 0.1]
# calculate team ratings for each match
home_team_rating = normalized_data.iloc[:, [2, 4 , 6, 8, 10, 10, 12]].mul(weights).sum(axis=1)
away_team_rating = normalized_data.iloc[:, [1, 3 , 5 ,7, 9, 11, 13]].mul(weights).sum(axis=1)





In [51]:
import pandas as pd

# combine home and away team ratings into one dataframe
homedf = df["HomeTeam"].dropna()
awaydf = df["AwayTeam"].dropna()
X = pd.concat([home_team_rating, away_team_rating], axis=1)

    

# convert FTR column into binary labels
df['FTR_binary'] = df['FTR'].apply(lambda x: 1 if x == 'H' else 0)

# split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['FTR_binary'], test_size=0.2, random_state=42)

# train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# evaluate model performance
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score,_= precision_recall_fscore_support(y_test, y_pred, average='binary')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)


Accuracy: 0.6666666666666666
Precision: 0.5991379310344828
Recall: 0.6150442477876106
F1-score: 0.6069868995633187


In [21]:
#saving the model
joblib.dump(logreg, 'logreg_model.h5')

['logreg_model.h5']

In [22]:

print(X)


             0         1
0     0.388014  0.084686
1    -0.108871  0.067898
2    -0.014034  0.202207
3    -0.066947  0.074083
4    -0.717268 -0.205503
...        ...       ...
2695 -0.469356  0.216305
2696  0.277834 -0.664023
2697  0.717898 -0.543074
2698  0.503573 -0.453227
2699 -0.701989  1.504037

[2700 rows x 2 columns]


In [52]:
homedf = df["HomeTeam"].dropna()
awaydf = df["AwayTeam"].dropna()
X2 = pd.concat([home_team_rating, away_team_rating,homedf, awaydf], axis=1)
X2.to_csv('ratings.csv', index=False)